In [2]:
 #pip install imap-tools

Note: you may need to restart the kernel to use updated packages.


In [9]:
from imap_tools import MailBox
import pandas as pd
from dotenv import load_dotenv
import os

In [10]:
load_dotenv("login.env")

True

In [11]:
login_address = os.environ.get("GMAIL_USER0")
password=os.environ.get("GMAIL_PASSWORD0")
folder='INBOX'

In [ ]:
with MailBox('imap.gmail.com').login(login_address, password, folder) as mailbox:
    allRows = []
    for msg in mailbox.fetch(mark_seen=False):

        print(f"{msg.uid} is processing")
        isRead = "Read" if '\\Seen' in msg.flags else "Unread"
        isFlagged = "True" if '\\Flagged' in msg.flags else "False"
        isEncrypted = "True" if 'ENCRYPTED' in msg.flags else "False"
        isSubscribed = False if msg.headers.get('list-unsubscribe', None) is None else True
        attachmentCount = len(msg.attachments)
        isPrimaryRecipient = True if login_address in [b.email for b in msg.to_values] else False
        isCC = True if login_address in [b.email for b in msg.cc_values] else False
        isBCC = True if login_address in [b.email for b in msg.bcc_values] else False
        MsgID = "A" + msg.uid
        attachSize = 0
        for att in msg.attachments:
            attachSize += att.size
        allRows.append(
            {"MessageID": MsgID, "Date": msg.date, "Sender": msg.from_,
             "Subject": msg.subject, "EmailSize": msg.size, "Status": isRead, "isSubscription": isSubscribed,
             "isEncrypted": isEncrypted, "isFlagged": isFlagged, "AttachmentCount": attachmentCount,
             "AttachmentSize": attachSize, "isPrimaryRecipient": isPrimaryRecipient, "NoOfToValues": len(msg.to_values),
             "isCC": isCC, "NoOfCCValues": len(msg.cc_values), "isBCC": isBCC})

In [12]:
df = pd.DataFrame(allRows, columns=["MessageID", "Date", "Sender", "Subject", "EmailSize", "Status",
                                        "isSubscription", "isEncrypted", "isFlagged", "AttachmentCount",
                                        "AttachmentSize", "isPrimaryRecipient", "NoOfToValues", "isCC", "NoOfCCValues",
                                        "isBCC"])

In [13]:
df.to_csv('MailOutput_latest.csv', index=False, encoding="utf-8-sig")